# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disasterResponse.db')
df = pd.read_sql_table('disasterResponseCleaned', engine)
X = df.message.values
Y = df[df.columns[4:]].values
category_names = list(df.columns[4:])
print(Y.shape)
print(category_names)

(26207, 35)
['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [4]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC()))
])

#cv = GridSearchCV(pipeline, param_grid=parameters, scoring='precision_samples', cv = 3)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
y_test
pipeline.fit(X_train,y_train)
y_pred=pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report, accuracy_score
print(y_pred.shape)
print(y_test.shape)
for i in range(y_test.shape[1]):
        print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(y_test[:,i], y_pred[:,i])))
print(classification_report(y_test, y_pred, target_names=category_names))
print(pipeline.named_steps)

(7863, 35)
(7863, 35)
                  related accuracy : 0.99
                  request accuracy : 0.89
                    offer accuracy : 0.99
              aid_related accuracy : 0.77
             medical_help accuracy : 0.93
         medical_products accuracy : 0.96
        search_and_rescue accuracy : 0.98
                 security accuracy : 0.98
                 military accuracy : 0.97
                    water accuracy : 0.96
                     food accuracy : 0.94
                  shelter accuracy : 0.94
                 clothing accuracy : 0.99
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.97
                other_aid accuracy : 0.87
   infrastructure_related accuracy : 0.93
                transport accuracy : 0.96
                buildings accuracy : 0.95
              electricity accuracy : 0.98
                    tools accuracy : 0.99
            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
parameters = {
    'clf__estimator__dual': [False],
    'clf__estimator__C':  [1,2],
    'tfidf__smooth_idf':[True]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [35]:
print(cv.best_params_)
print(classification_report(y_test, y_pred, target_names = category_names))

{'clf__estimator__C': 1, 'clf__estimator__dual': False, 'tfidf__smooth_idf': True}
                        precision    recall  f1-score   support

               related       0.64      0.20      0.30        46
               request       0.76      0.58      0.66      1328
                 offer       0.00      0.00      0.00        37
           aid_related       0.72      0.68      0.70      3260
          medical_help       0.64      0.30      0.41       627
      medical_products       0.70      0.31      0.43       410
     search_and_rescue       0.66      0.16      0.26       202
              security       0.29      0.04      0.06       137
              military       0.58      0.31      0.40       255
                 water       0.76      0.62      0.68       509
                  food       0.80      0.68      0.74       888
               shelter       0.74      0.54      0.62       669
              clothing       0.68      0.43      0.53       121
                 mon

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:
# Using RandomForest for classification task
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
pipeline.fit(X_train,y_train)
y_pred=pipeline.predict(X_test)
for i in range(y_test.shape[1]):
        print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(y_test[:,i], y_pred[:,i])))
print(classification_report(y_test, y_pred, target_names=category_names))

                  related accuracy : 0.99
                  request accuracy : 0.88
                    offer accuracy : 1.00
              aid_related accuracy : 0.74
             medical_help accuracy : 0.92
         medical_products accuracy : 0.95
        search_and_rescue accuracy : 0.97
                 security accuracy : 0.98
                 military accuracy : 0.97
                    water accuracy : 0.96
                     food accuracy : 0.92
                  shelter accuracy : 0.93
                 clothing accuracy : 0.98
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.96
                other_aid accuracy : 0.87
   infrastructure_related accuracy : 0.94
                transport accuracy : 0.96
                buildings accuracy : 0.95
              electricity accuracy : 0.98
                    tools accuracy : 0.99
                hospitals accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# Using Logistic Regression for classification task
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
pipeline.fit(X_train,y_train)
y_pred=pipeline.predict(X_test)
for i in range(y_test.shape[1]):
        print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(y_test[:,i], y_pred[:,i])))
print(classification_report(y_test, y_pred, target_names=category_names))

                  related accuracy : 0.99
                  request accuracy : 0.90
                    offer accuracy : 1.00
              aid_related accuracy : 0.78
             medical_help accuracy : 0.92
         medical_products accuracy : 0.96
        search_and_rescue accuracy : 0.97
                 security accuracy : 0.98
                 military accuracy : 0.97
                    water accuracy : 0.96
                     food accuracy : 0.94
                  shelter accuracy : 0.94
                 clothing accuracy : 0.99
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.96
                other_aid accuracy : 0.87
   infrastructure_related accuracy : 0.94
                transport accuracy : 0.96
                buildings accuracy : 0.95
              electricity accuracy : 0.98
                    tools accuracy : 0.99
                hospitals accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.